<a href="https://colab.research.google.com/github/Jerry6coding/100days/blob/master/NLP_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

In [35]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [22]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ML_data/NLP_test/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_data/NLP_test/test.csv')
sample = pd.read_csv('/content/drive/MyDrive/ML_data/NLP_test/sample_submission.csv')

In [36]:
train

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [38]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [40]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [41]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


In [42]:
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [46]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2021-03-13 23:44:17--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-13 23:44:17--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-13 23:44:18--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [47]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [48]:
!ls
!pwd

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [56]:
embeddings_index = {}
f = open('/content/glove.6B.300d.txt')
for line in tqdm(f, desc="Loading..."):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


Loading...: 0it [00:00, ?it/s]
Loading...: 1348it [00:00, 13473.43it/s]
Loading...: 2610it [00:00, 13203.33it/s]
Loading...: 3908it [00:00, 13132.86it/s]
Loading...: 5151it [00:00, 12912.51it/s]
Loading...: 6323it [00:00, 12527.11it/s]
Loading...: 7576it [00:00, 12527.46it/s]
Loading...: 8846it [00:00, 12574.62it/s]
Loading...: 10112it [00:00, 12599.48it/s]
Loading...: 11435it [00:00, 12780.57it/s]
Loading...: 12783it [00:01, 12982.57it/s]
Loading...: 14087it [00:01, 12996.97it/s]
Loading...: 15384it [00:01, 12988.24it/s]
Loading...: 16666it [00:01, 12804.68it/s]
Loading...: 17985it [00:01, 12917.80it/s]
Loading...: 19332it [00:01, 13076.67it/s]
Loading...: 20672it [00:01, 13171.65it/s]
Loading...: 22003it [00:01, 13212.69it/s]
Loading...: 23322it [00:01, 12889.87it/s]
Loading...: 24627it [00:01, 12935.72it/s]
Loading...: 25921it [00:02, 12897.26it/s]
Loading...: 27211it [00:02, 12842.59it/s]
Loading...: 28496it [00:02, 12676.46it/s]
Loading...: 29785it [00:02, 12739.42it/s]
Loading..

Found 400000 word vectors.


In [57]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


100%|██████████| 25943/25943 [00:00<00:00, 323654.09it/s]
